# ResNeSt


<img src="https://pytorch.org/assets/images/resnest.jpg" alt="alt" width="50%"/>

Convolution neural networks have largely dominated the computer vision domain, but in the last few years, feature-map attention architectures like SE-Net and SK-Net have started to assert dominance. In their paper “ResNeSt: Split-Attention Networks”,  Hang Zhang,  Chongruo Wu, et al. proposed a new ResNet variant that combines the best of both worlds. The ResNeSt architecture leverages the channel-wise attention with multi-path representation into a single unified Split-Attention block. It learns cross-channel feature correlations while preserving independent representation in the meta structure.

To read more about its architecture, please refer [this](https://analyticsindiamag.com/guide-to-resnest-a-better-resnet-with-the-same-costs/) article.

# Image Classification with ResNeSt

  Install PyTorch and fvcore.

In [ ]:
!python -m pip install pip --upgrade --user -q --no-warn-script-location
!python -m pip install numpy pandas seaborn matplotlib scipy statsmodels sklearn tensorflow keras opencv-python pillow scikit-image fvcore torch torchvision --user -q --no-warn-script-location

import IPython
IPython.Application.instance().kernel.do_shutdown(True)


Download the pre-trained ResNeSt model from Torch hub and set it eval mode for making inferences.

In [ ]:
import torch
# get list of models
torch.hub.list('zhanghang1989/ResNeSt', force_reload=True)

All pre-trained models expect input images normalized in the same way,
i.e. mini-batches of 3-channel RGB images of shape `(3 x H x W)`, where `H` and `W` are expected to be at least `224`.
The images have to be loaded in to a range of `[0, 1]` and then normalized using `mean = [0.485, 0.456, 0.406]`
and `std = [0.229, 0.224, 0.225]`.

In [ ]:
# load pretrained models, using ResNeSt-50 as an example
model = torch.hub.load('zhanghang1989/ResNeSt', 'resnest50', pretrained=True)
model.eval()

Get the image(s) for making inferences.

In [ ]:
# !wget https://raw.githubusercontent.com/EliSchwartz/imagenet-sample-images/master/n01491361_tiger_shark.JPEG

In [ ]:
# Download an example image
filename = "n01491361_tiger_shark.JPEG"

In [ ]:
from PIL import Image
from torchvision import transforms
input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)
    
# Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
print(output)

In [ ]:
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
probabilities = torch.nn.functional.softmax(output[0], dim=0)
print(probabilities)

In [ ]:
# Download ImageNet labels
# !wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

Apply softmax on the confidence scores of the 1000 ImageNet classes to get the probabilities. Print the top 3(k) categories. 

In [ ]:
# Read the categories
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
# Show top categories per image
top3_prob, top3_catid = torch.topk(probabilities, 3)
for i in range(top3_prob.size(0)):
    print(categories[top3_catid[i]], top3_prob[i].item())